In [0]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11864113949564533286
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15351628376638560098
physical_device_desc: "device: XLA_CPU device"
]


In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


[]

In [0]:
import numpy as np
import random
import json
from glob import glob
from scipy import ndimage
from keras.models import Model,load_model
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dropout,GaussianNoise, Input,Activation
from keras.layers.normalization import BatchNormalization
from keras.layers import  Conv2DTranspose,UpSampling2D,concatenate,add
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import  ModelCheckpoint,Callback,LearningRateScheduler
import keras.backend as K


Using TensorFlow backend.


# Unet Model

In [0]:
K.set_image_data_format("channels_last")

 #u-net model
class Unet_model(object):
    
    def __init__(self,img_shape,load_model_weights=None):
        self.img_shape=img_shape
        self.load_model_weights=load_model_weights
        self.model =self.compile_unet()
        
    
    def compile_unet(self):
        """
        compile the U-net model
        """
        i = Input(shape=self.img_shape)
        #add gaussian noise to the first layer to combat overfitting
        i_=GaussianNoise(0.01)(i)

        i_ = Conv2D(64, 2, padding='same',data_format = 'channels_last')(i_)
        out=self.unet(inputs=i_)
        model = Model(input=i, output=out)

        sgd = SGD(lr=0.08, momentum=0.9, decay=5e-6, nesterov=False)
        model.compile(loss=gen_dice_loss, optimizer=sgd, metrics=[dice_whole_metric,dice_en_metric,dice_core_metric])
       
        #load weights if set for prediction
        if self.load_model_weights is not None:
            model.load_weights(self.load_model_weights)
        print("Model compilation properly done")
        return model


    def unet(self,inputs, nb_classes=4, start_ch=64, depth=3, inc_rate=2. ,activation='relu', dropout=0.0, batchnorm=True, upconv=True,format_='channels_last'):
        """
        the actual u-net architecture
        """
        o = self.level_block(inputs,start_ch, depth, inc_rate,activation, dropout, batchnorm, upconv,format_)
        o = BatchNormalization()(o) 
        #o =  Activation('relu')(o)
        o=PReLU(shared_axes=[1, 2])(o)
        o = Conv2D(nb_classes, 1, padding='same',data_format = format_)(o)
        o = Activation('softmax')(o)
        return o



    def level_block(self,m, dim, depth, inc, acti, do, bn, up,format_="channels_last"):
        if depth > 0:
            n = self.res_block_enc(m,0.0,dim,acti, bn,format_)
            #using strided 2D conv for donwsampling
            m = Conv2D(int(inc*dim), 2,strides=2, padding='same',data_format = format_)(n)
            m = self.level_block(m,int(inc*dim), depth-1, inc, acti, do, bn, up )
            if up:
                m = UpSampling2D(size=(2, 2),data_format = format_)(m)
                m = Conv2D(dim, 2, padding='same',data_format = format_)(m)
            else:
                m = Conv2DTranspose(dim, 3, strides=2,padding='same',data_format = format_)(m)
            n=concatenate([n,m])
            #the decoding path
            m = self.res_block_dec(n, 0.0,dim, acti, bn, format_)
        else:
            m = self.res_block_enc(m, 0.0,dim, acti, bn, format_)
        
        return m

  
   
    def res_block_enc(self,m, drpout,dim,acti, bn,format_="channels_last"):
        
        """
        the encoding unit which a residual block
        """
        n = BatchNormalization()(m) if bn else n
        #n=  Activation(acti)(n)
        n=PReLU(shared_axes=[1, 2])(n)
        n = Conv2D(dim, 3, padding='same',data_format = format_)(n)
                
        n = BatchNormalization()(n) if bn else n
        #n=  Activation(acti)(n)
        n=PReLU(shared_axes=[1, 2])(n)
        n = Conv2D(dim, 3, padding='same',data_format =format_ )(n)

        n=add([m,n]) 
        
        return  n 



    def res_block_dec(self,m, drpout,dim,acti, bn,format_="channels_last"):

        """
        the decoding unit which a residual block
        """
         
        n = BatchNormalization()(m) if bn else n
        #n=  Activation(acti)(n)
        n=PReLU(shared_axes=[1, 2])(n)
        n = Conv2D(dim, 3, padding='same',data_format = format_)(n)

        n = BatchNormalization()(n) if bn else n
        #n=  Activation(acti)(n)
        n=PReLU(shared_axes=[1, 2])(n)
        n = Conv2D(dim, 3, padding='same',data_format =format_ )(n)
        
        Save = Conv2D(dim, 1, padding='same',data_format = format_,use_bias=False)(m) 
        n=add([Save,n]) 
        
        return  n  
    print("")

# Losses

In [0]:
def dice(y_true, y_pred):
    #computes the dice score on two tensors

    sum_p=K.sum(y_pred,axis=0)
    sum_r=K.sum(y_true,axis=0)
    sum_pr=K.sum(y_true * y_pred,axis=0)
    dice_numerator =2*sum_pr
    dice_denominator =sum_r+sum_p
    dice_score =(dice_numerator+K.epsilon() )/(dice_denominator+K.epsilon())
    
    return dice_score
  
def dice_core_metric(y_true, y_pred):
    ##computes the dice for the core region

    y_true_f = K.reshape(y_true,shape=(-1,4))
    y_pred_f = K.reshape(y_pred,shape=(-1,4))
    
    #workaround for tf
    #y_core=K.sum(tf.gather(y_true_f, [1,3],axis =1),axis=1)
    #p_core=K.sum(tf.gather(y_pred_f, [1,3],axis =1),axis=1)
    
    y_core_1 = y_true_f[:,1]
    y_core_3 = y_true_f[:,3]
    y_core = y_core_1 + y_core_3 
    p_core_1 = y_pred_f[:,1]
    p_core_3 = y_pred_f[:,3]
    p_core = p_core_1 + p_core_3 
    dice_core=dice(y_core,p_core)
    return dice_core


def dice_whole_metric(y_true, y_pred):
    #computes the dice for the whole tumor

    y_true_f = K.reshape(y_true,shape=(-1,4))
    y_pred_f = K.reshape(y_pred,shape=(-1,4))
    y_whole=K.sum(y_true_f[:,1:],axis=1)
    p_whole=K.sum(y_pred_f[:,1:],axis=1)
    dice_whole=dice(y_whole,p_whole)
    
    return dice_whole

def dice_en_metric(y_true, y_pred):
    #computes the dice for the enhancing region

    y_true_f = K.reshape(y_true,shape=(-1,4))
    y_pred_f = K.reshape(y_pred,shape=(-1,4))
    y_enh=y_true_f[:,-1]
    p_enh=y_pred_f[:,-1]
    dice_en=dice(y_enh,p_enh)
    
    return dice_en





def weighted_log_loss(y_true, y_pred):
    # scale predictions so that the class probas of each sample sum to 1
    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
    # clip to prevent NaN's and Inf's
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # weights are assigned in this order : normal,necrotic,edema,enhancing 
    weights=np.array([1,5,2,4])
    weights = K.variable(weights)
    loss = y_true * K.log(y_pred) * weights
    loss = K.mean(-K.sum(loss, -1))
    
    return loss

def gen_dice_loss(y_true, y_pred):
    '''
    computes the sum of two losses : generalised dice loss and weighted cross entropy
    '''

    #generalised dice score is calculated as in this paper : https://arxiv.org/pdf/1707.03237
    y_true_f = K.reshape(y_true,shape=(-1,4))
    y_pred_f = K.reshape(y_pred,shape=(-1,4))
    sum_p=K.sum(y_pred_f,axis=-2)
    sum_r=K.sum(y_true_f,axis=-2)
    sum_pr=K.sum(y_true_f * y_pred_f,axis=-2)
    weights=K.pow(K.square(sum_r)+K.epsilon(),-1)
    generalised_dice_numerator =2*K.sum(weights*sum_pr)
    generalised_dice_denominator =K.sum(weights*(sum_r+sum_p))
    generalised_dice_score =generalised_dice_numerator /generalised_dice_denominator
    GDL=1-generalised_dice_score
    del sum_p,sum_r,sum_pr,weights
    print(GDL+weighted_log_loss(y_true,y_pred))
    return GDL+weighted_log_loss(y_true,y_pred)

# New Section

In [0]:
class SGDLearningRateTracker(Callback):
    def on_epoch_begin(self, epoch, logs={}):
        optimizer = self.model.optimizer
        lr = K.get_value(optimizer.lr)
        decay = K.get_value(optimizer.decay)
        lr=lr/10
        decay=decay*10
        K.set_value(optimizer.lr, lr)
        K.set_value(optimizer.decay, decay)
        print('LR changed to:',lr)
        print('Decay changed to:',decay)



class Training(object):
    

    def __init__(self, batch_size,nb_epoch,load_model_resume_training=None):

        self.batch_size = batch_size
        self.nb_epoch = nb_epoch

        #loading model from path to resume previous training without recompiling the whole model
        if load_model_resume_training is not None:
            self.model =load_model(load_model_resume_training,custom_objects={'gen_dice_loss': gen_dice_loss,'dice_whole_metric':dice_whole_metric,'dice_core_metric':dice_core_metric,'dice_en_metric':dice_en_metric})
            print("pre-trained model loaded!")
        else:
            unet =Unet_model(img_shape=(128,128,4))
            self.model=unet.model
            print("U-net CNN compiled!")

                    
    def fit_unet(self,X33_train,Y_train,X_patches_valid=None,Y_labels_valid=None):

        train_generator=self.img_msk_gen(X33_train,Y_train,9999)
        checkpointer = ModelCheckpoint(filepath='drive/My Drive/Brain Tumor Segmentation/ResUnet.{epoch:02d}_{val_loss:.3f}.hdf5', verbose=1)
        self.model.fit_generator(train_generator,steps_per_epoch=len(X33_train)//self.batch_size,epochs=self.nb_epoch, validation_data=(X_patches_valid,Y_labels_valid),verbose=1, callbacks = [checkpointer,SGDLearningRateTracker()])
        #self.model.fit(X33_train,Y_train, epochs=self.nb_epoch,batch_size=self.batch_size,validation_data=(X_patches_valid,Y_labels_valid),verbose=1, callbacks = [checkpointer,SGDLearningRateTracker()])

    def img_msk_gen(self,X33_train,Y_train,seed):

        '''
        a custom generator that performs data augmentation on both patches and their corresponding targets (masks)
        '''
        datagen = ImageDataGenerator(horizontal_flip=True,data_format="channels_last")
        datagen_msk = ImageDataGenerator(horizontal_flip=True,data_format="channels_last")
        image_generator = datagen.flow(X33_train,batch_size=4,seed=seed)
        y_generator = datagen_msk.flow(Y_train,batch_size=4,seed=seed)
        while True:
            yield(image_generator.next(), y_generator.next())

    def save_model(model_name):
        '''
        INPUT string 'model_name': path where to save model and weights, without extension
        Saves current model as json and weights as h5df file
        '''
        model_tosave = '{}.json'.format(model_name)
        weights = '{}.hdf5'.format(model_name)
        json_string = self.to_json()
        self.model.save_weights(weights)
        with open(model_tosave, 'w') as f:
            json.dump(json_string, f)
        print ('Model saved.')
  

    def load_model(self, model_name):
        '''
        Load a model
        INPUT  (1) string 'model_name': filepath to model and weights, not including extension
        OUTPUT: Model with loaded weights. can fit on model using loaded_model=True in fit_model method
        '''
        print ('Loading model {}'.format(model_name))
        model_toload = '{}.json'.format(model_name)
        weights = '{}.hdf5'.format(model_name)
        with open(model_toload) as f:
            m = next(f)
        model_comp = model_from_json(json.loads(m))
        model_comp.load_weights(weights)
        print ('Model loaded.')
        self.model = model_comp
        return model_comp




In [0]:
if __name__ == "__main__":
    #set arguments

    #reload already trained model to resume training
    model_to_load="drive/My Drive/Brain Tumor Segmentation/ResUnet.01_0.728.hdf5" 
    #save=None

    #compile the model
    brain_seg = Training(batch_size=4,nb_epoch=1,load_model_resume_training=model_to_load)

    print("number of trainabale parameters:",brain_seg.model.count_params())
    #print(brain_seg.model.summary())
    #plot(brain_seg.model, to_file='model_architecture.png', show_shapes=True)

    #load data from disk
    
    
    print("loading patches done\n")

    # fit model
    brain_seg.fit_unet(X_patches,Y_labels,X_patches_valid,Y_labels_valid)

Instructions for updating:
Colocations handled automatically by placer.
Tensor("loss/activation_1_loss/add_2:0", shape=(), dtype=float32)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
pre-trained model loaded!
number of trainabale parameters: 10159748
loading patches done

Epoch 1/1
LR changed to: 7.999999215826392e-05
Decay changed to: 0.004999999655410647
   1/1752 [..............................] - ETA: 7:23:37 - loss: 1.0215 - dice_whole_metric: 0.7900 - dice_en_metric: 3.2471e-09 - dice_core_metric: 1.2534e-04

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
Y = np.load("drive/My Drive/Brain Tumor Segmentation/Training Array/Y_labels_128.npy").astype(np.uint8) #Don't use /content while loading a file
 

In [0]:
X = np.load("drive/My Drive/Brain Tumor Segmentation/Training Array/X_patches_128.npy").astype(np.float32)

In [0]:
from sklearn.model_selection import train_test_split
X_patches,X_patches_valid,Y_labels,Y_labels_valid = train_test_split(X, Y, test_size=0.20, random_state=42)

In [0]:
X.shape

(8760, 128, 128, 4)